<a href="https://colab.research.google.com/github/chiransiriwardhana/FYP_Conditional_GAN/blob/main/Conditional_Generative_Adversarial_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

tpu_address = TF_MASTER

In [ ]:
import tensorflow as tf
# %tensorflow_version 2.2.0
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(resolver)

tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
print("Number of devices: ", len(tf.config.list_logical_devices('TPU')))

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.89.54.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.89.54.66:8470


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os

len(os.listdir('/content/drive/MyDrive/binarized_images/'))

In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np


def add_gaussian_noise(img):

    # data_ = normalise(data, (-1, 1), data_range)
    # noisy = data_ + np.random.normal(mean, stdev, data.shape)
    # noisy = np.clip(noisy, -1, 1) if clip else noisy
    # return normalise(noisy, data_range, (-1, 1))

    VARIABILITY = 50
    deviation  = VARIABILITY * random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

    # return noisy

In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
import math


def normalize(data, new_range=(-1, 1), current_range=None, axis=None):
  
    s = new_range[1] - new_range[0]
    if current_range is not None:
        mins = current_range[0]
        maxs = current_range[1]
    elif axis is not None:
        mins = np.nanmin(data, axis=axis, keepdims=True)
        maxs = np.nanmax(data, axis=axis, keepdims=True)   
    else:
        mins = data.min()
        maxs = data.max() 
    return s * (data - mins) / (maxs - mins) + new_range[0]

In [ ]:
import cv2
import random

folder_name = os.listdir('/content/drive/MyDrive/binarized_images/')
X = list()
y = list()
for i in range(len(folder_name)):
  img = cv2.imread('/content/drive/MyDrive/binarized_images/'+str(i)+".jpg")
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, dsize=(150, 150), interpolation=cv2.INTER_CUBIC)

  img_ = img.astype('float32')
  y_img = normalize(img_)
  y.append(y_img)
  noisy = normalize(add_gaussian_noise(img_))
  X.append(noisy)

In [ ]:
test_X = list()
for i in range(len(X)):
  if (i == 431):
    break
  else:
    test_X.append(X[i])
    X.pop(i)

In [ ]:
test_y = list()
for i in range(len(y)):
  if (i == 431):
    break
  else:
    test_y.append(y[i])
    y.pop(i)

In [ ]:
train_X = X 
train_y = y

train_X = np.array(train_X)


In [ ]:
train_y = np.array(train_y)


In [ ]:
test_X = np.array(test_X)

In [ ]:

test_y = np.array(test_y)

In [ ]:
print("train_X shape: ", train_X.shape)
print("train_y shape: ", train_y.shape)
print("test_X shape: ", test_X.shape)
print("test_y shape", test_y.shape)

In [ ]:
from matplotlib import pyplot as plt
img_f = train_X[6]
plt.axis("off")
plt.imshow(cv2.cvtColor(img_f, cv2.COLOR_BGR2RGB), interpolation='nearest')
plt.show()

In [ ]:
img_f = train_y[6]
plt.axis("off")
plt.imshow(cv2.cvtColor(img_f, cv2.COLOR_BGR2RGB), interpolation='nearest')
plt.show()

In [ ]:
train_X = np.expand_dims(train_X, axis=3)
train_y = np.expand_dims(train_y, axis=3)
test_X = np.expand_dims(test_X, axis=3)
test_y = np.expand_dims(test_y, axis=3)

In [ ]:
print("train_X shape: ", train_X.shape)
print("train_y shape: ", train_y.shape)
print("test_X shape: ", test_X.shape)
print("test_y shape", test_y.shape)

In [ ]:


from os.path import join

class ConfigCGAN:
    """
    Configuration parameters for the Conditional GAN
    """
    # Dimensions
    raw_size = 150
    adjust_size = 150
    train_size = 150
    channels = 1
    base_number_of_filters = 64
    kernel_size = (3, 3)
    strides = (2, 2)

    # Fixed model parameters
    leak = 0.2
    dropout_rate = 0.5

    # Hyperparameters
    learning_rate = 2e-4
    beta1 = 0.5
    max_epoch = 300
    L1_lambda = 100

    # Data
    buffer_size = 1500
    batch_size = 50

    # Data storage
    save_per_epoch = 5
    exp_name = 'noise_gan'
    data_path = join('/content/drive/MyDrive/out', exp_name, 'data')
    model_path = join('/content/drive/MyDrive/out', exp_name, 'model')
    results_path = join('/content/drive/MyDrive/out', exp_name, 'results')

In [ ]:
config = ConfigCGAN()

In [ ]:
def next_power_2(n):
    """
    Compute the nearest power of 2 greater than n.
    Arguments:
        n: integer.
    """
    count = 0
    # If it is a non-zero power of 2, return it
    if n and not (n & (n - 1)): 
        return n 
    # Keep dividing n by 2 until it is 0
    while n != 0:  
        n >>= 1
        count += 1
    # Result is 2 to the power of divisions taken
    return 1 << count

In [ ]:
def padding_power_2(shape):
    """
    Get the padding required to change the given shape to a square power 
    of 2 in each dimension.
    Arguments:
        shape: tuple of 2 ints. The original shape.
    """
    padded_size = next_power_2(max(shape))
    return ((padded_size - shape[0])//2, (padded_size - shape[1])//2)


In [ ]:
def mse(x1, x2, norm=2):
    return tf.reduce_mean(tf.square((x1 - x2) / norm))


In [ ]:
def rmse(x1, x2, norm=2):
    return tf.sqrt(mse(x1, x2, norm))

In [ ]:
def psnr(x1, x2, max_diff=1):
    return 20. * tf.math.log(max_diff / rmse(x1, x2)) / tf.math.log(10.)


In [ ]:
def make_generator_model():
    f = config.base_number_of_filters
    k = config.kernel_size
    s = config.strides
    sz = config.train_size
    c = config.channels
    pad = padding_power_2((sz, sz))

    if sz <= 128:
        raise RuntimeError("Input size must be larger than 128 for this U-Net model")

    inputs = tf.keras.layers.Input((sz, sz, c), name="ginput")
    # scaled_input = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)(inputs)
    inputs_pad = tf.keras.layers.ZeroPadding2D(pad, name="gpad")(inputs)

    # Encoder layers
    # Input is sz x sz x c
    ge1 = tf.keras.layers.Conv2D(f, k, s, padding="same", name="geconv1")(inputs_pad)
    # Input is sz2 x sz2 x f
    ge2 = tf.keras.layers.LeakyReLU(config.leak, name="geact1")(ge1)
    ge2 = tf.keras.layers.Conv2D(2*f, k, s, padding="same", name="geconv2")(ge2)
    ge2 = tf.keras.layers.BatchNormalization(name="gebn2")(ge2)
    # Input is sz4 x sz4 x 2f
    ge3 = tf.keras.layers.LeakyReLU(config.leak, name="geact2")(ge2)
    ge3 = tf.keras.layers.Conv2D(4*f, k, s, padding="same", name="geconv3")(ge3)
    ge3 = tf.keras.layers.BatchNormalization(name="gebn3")(ge3)
    # Input is sz8 x sz8 x 4f
    ge4 = tf.keras.layers.LeakyReLU(config.leak, name="geact3")(ge3)
    ge4 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv4")(ge4)
    ge4 = tf.keras.layers.BatchNormalization(name="gebn4")(ge4)
    # Input is sz16 x sz16 x 8f
    ge5 = tf.keras.layers.LeakyReLU(config.leak, name="geact4")(ge4)
    ge5 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv5")(ge5)
    ge5 = tf.keras.layers.BatchNormalization(name="gebn5")(ge5)
    # Input is sz32 x sz32 x 8f
    ge6 = tf.keras.layers.LeakyReLU(config.leak, name="geact5")(ge5)
    ge6 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv6")(ge6)
    ge6 = tf.keras.layers.BatchNormalization(name="gebn6")(ge6)
    # Input is sz64 x sz64 x 8f
    ge7 = tf.keras.layers.LeakyReLU(config.leak, name="geact6")(ge6)
    ge7 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv7")(ge7)
    ge7 = tf.keras.layers.BatchNormalization(name="gebn7")(ge7)
    # Input is sz128 x sz128 x 8f
    ge8 = tf.keras.layers.LeakyReLU(config.leak, name="geact7")(ge7)
    ge8 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv8")(ge8)
    ge8 = tf.keras.layers.BatchNormalization(name="gebn8")(ge8)
    # Input is sz256 x sz256 x 8f

    # Decoder layers with skip connections
    gd1 = tf.keras.layers.LeakyReLU(0.0, name="geact8")(ge8)
    gd1 = tf.keras.layers.Conv2DTranspose(8*f, k, s, padding="same", name="gdconv1")(gd1)
    gd1 = tf.keras.layers.BatchNormalization(name="gdbn1")(gd1)
    gd1 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop1")(gd1)
    # Input is sz128 x sz128 x 8f
    gd1 = tf.keras.layers.concatenate([gd1, ge7], axis=3, name="gdcat1")
    gd2 = tf.keras.layers.LeakyReLU(0.0, name="gdact1")(gd1)
    gd2 = tf.keras.layers.Conv2DTranspose(8*f, k, s, padding="same", name="gdconv2")(gd2)
    gd2 = tf.keras.layers.BatchNormalization(name="gdbn2")(gd2)
    gd2 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop2")(gd2)
    # Input is sz64 x sz64 x 8f
    gd2 = tf.keras.layers.concatenate([gd2, ge6], axis=3, name="gdcat2")
    gd3 = tf.keras.layers.LeakyReLU(0.0, name="gdact2")(gd2)
    gd3 = tf.keras.layers.Conv2DTranspose(8*f, k, s, padding="same", name="gdconv3")(gd3)
    gd3 = tf.keras.layers.BatchNormalization(name="gdbn3")(gd3)
    gd3 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop3")(gd3)
    # Input is sz32 x sz32 x 8f
    gd3 = tf.keras.layers.concatenate([gd3, ge5], axis=3, name="gdcat3")
    gd4 = tf.keras.layers.LeakyReLU(0.0, name="gdact3")(gd3)
    gd4 = tf.keras.layers.Conv2DTranspose(8*f, k, s, padding="same", name="gdconv4")(gd4)
    gd4 = tf.keras.layers.BatchNormalization(name="gdbn4")(gd4)
    # Input is sz16 x sz16 x 8f
    gd4 = tf.keras.layers.concatenate([gd4, ge4], axis=3, name="gdcat4")
    gd5 = tf.keras.layers.LeakyReLU(0.0, name="gdact4")(gd4)
    gd5 = tf.keras.layers.Conv2DTranspose(4*f, k, s, padding="same", name="gdconv5")(gd5)
    gd5 = tf.keras.layers.BatchNormalization(name="gdbn5")(gd5)
    gd5 = tf.keras.layers.concatenate([gd5, ge3], axis=3, name="gdcat5")
    # Input is sz8 x sz8 x 4f
    gd6 = tf.keras.layers.LeakyReLU(0.0, name="gdact5")(gd5)
    gd6 = tf.keras.layers.Conv2DTranspose(2*f, k, s, padding="same", name="gdconv6")(gd6)
    gd6 = tf.keras.layers.BatchNormalization(name="gdbn6")(gd6)
    # Input is sz4 x sz4 x 2f
    gd6 = tf.keras.layers.concatenate([gd6, ge2], axis=3, name="gdcat6")
    gd7 = tf.keras.layers.LeakyReLU(0.0, name="gdact6")(gd6)
    gd7 = tf.keras.layers.Conv2DTranspose(f, k, s, padding="same", name="gdconv7")(gd7)
    gd7 = tf.keras.layers.BatchNormalization(name="gdbn7")(gd7)
    # Input is sz2 x sz2 x f
    gd7 = tf.keras.layers.concatenate([gd7, ge1], axis=3, name="gdcat7")
    gd8 = tf.keras.layers.LeakyReLU(0.0)(gd7)
    gd8 = tf.keras.layers.Conv2DTranspose(c, k, s, padding="same", activation="tanh", 
                          name="gdconvout")(gd8)
    # Input is sz x sz x nc
    outputs = tf.keras.layers.Cropping2D(pad, name="gcrop")(gd8)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="cond_gen")
  
    return model


def make_generator_model_small():
    f = config.base_number_of_filters
    k = config.kernel_size
    s = config.strides
    sz = config.train_size
    c = config.channels
    pad = padding_power_2((sz, sz))

    inputs = tf.keras.layers.Input((sz, sz, c), name="ginput")
    # scaled_input = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)(inputs)
    inputs_pad = tf.keras.layers.ZeroPadding2D(pad, name="gpad")(inputs)

    # Encoder layers
    ge1 = tf.keras.layers.Conv2D(f, k, s, padding="same", name="geconv1")(inputs_pad)
    
    ge2 = tf.keras.layers.LeakyReLU(config.leak, name="geact1")(ge1)
    ge2 = tf.keras.layers.Conv2D(2*f, k, s, padding="same", name="geconv2")(ge2)
    ge2 = tf.keras.layers.BatchNormalization(name="gebn2")(ge2)
    
    ge3 = tf.keras.layers.LeakyReLU(config.leak, name="geact2")(ge2)
    ge3 = tf.keras.layers.Conv2D(4*f, k, s, padding="same", name="geconv3")(ge3)
    ge3 = tf.keras.layers.BatchNormalization(name="gebn3")(ge3)

    ge4 = tf.keras.layers.LeakyReLU(config.leak, name="geact3")(ge3)
    ge4 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="geconv4")(ge4)
    ge4 = tf.keras.layers.BatchNormalization(name="gebn4")(ge4)

    # Decoder layers with skip connections
    gd1 = tf.keras.layers.LeakyReLU(0.0, name="geact4")(ge4)
    # TODO not sure if dimensions need specifying
    gd1 = tf.keras.layers.Conv2DTranspose(4*f, k, s, padding="same", name="gdconv1")(gd1)
    gd1 = tf.keras.layers.BatchNormalization(name="gdbn1")(gd1)
    gd1 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop1")(gd1)
    gd1 = tf.keras.layers.concatenate([gd1, ge3], axis=3, name="gdcat1")

    gd2 = tf.keras.layers.LeakyReLU(0.0, name="gdact1")(gd1)
    gd2 = tf.keras.layers.Conv2DTranspose(2*f, k, s, padding="same", name="gdconv2")(gd2)
    gd2 = tf.keras.layers.BatchNormalization(name="gdbn2")(gd2)
    gd2 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop2")(gd2)
    gd2 = tf.keras.layers.concatenate([gd2, ge2], axis=3, name="gdcat2")

    gd3 = tf.keras.layers.LeakyReLU(0.0, name="gdact2")(gd2)
    gd3 = tf.keras.layers.Conv2DTranspose(f, k, s, padding="same", name="gdconv3")(gd3)
    gd3 = tf.keras.layers.BatchNormalization(name="gdbn3")(gd3)
    gd3 = tf.keras.layers.Dropout(config.dropout_rate, name="gddrop3")(gd3)
    gd3 = tf.keras.layers.concatenate([gd3, ge1], axis=3, name="gdcat3")

    gd4 = tf.keras.layers.LeakyReLU(0.0)(gd3)
    gd4 = tf.keras.layers.Conv2DTranspose(c, k, s, padding="same", activation="tanh", 
                          name="gdconvout")(gd4)
    
    outputs = tf.keras.layers.Cropping2D(pad, name="gcrop")(gd4)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="cond_gen")
  
    return model


In [ ]:
generator = make_generator_model_small()
tf.keras.utils.plot_model(generator, show_shapes=True)

In [ ]:
generator.summary()

In [ ]:
generator_large = make_generator_model()

In [ ]:
tf.keras.utils.plot_model(generator_large, show_shapes=True)

In [ ]:
generator_large.summary()

In [ ]:
def make_discriminator_model():
    f = config.base_number_of_filters
    k = config.kernel_size
    s = config.strides
    sz = config.train_size
    c = config.channels

    inputs = tf.keras.layers.Input((sz, sz, c), name="dinput")
    # scaled_input = tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)(inputs)
    d0 = tf.keras.layers.Conv2D(f, k, s, padding="same", name="dconv0")(inputs)
    d0 = tf.keras.layers.LeakyReLU(config.leak, name="dact0")(d0)
    
    d1 = tf.keras.layers.Conv2D(2*f, k, s, padding="same", name="dconv1")(d0)
    d1 = tf.keras.layers.BatchNormalization(name="dbn1")(d1)
    d1 = tf.keras.layers.LeakyReLU(config.leak, name="dact1")(d1)

    d2 = tf.keras.layers.Conv2D(4*f, k, s, padding="same", name="dconv2")(d1)
    d2 = tf.keras.layers.BatchNormalization(name="dbn2")(d2)
    d2 = tf.keras.layers.LeakyReLU(config.leak, name="dact2")(d2)

    d3 = tf.keras.layers.Conv2D(8*f, k, s, padding="same", name="dconv3")(d2)
    d3 = tf.keras.layers.BatchNormalization(name="dbn3")(d3)
    d3 = tf.keras.layers.LeakyReLU(config.leak, name="dact3")(d3)

    d4 = tf.keras.layers.Flatten(name="dflatout")(d3)

    outputs = tf.keras.layers.Dense(1, name="ddenseout")(d4)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="cond_dsc")

    return model

In [ ]:
discriminator = make_discriminator_model()
tf.keras.utils.plot_model(discriminator, show_shapes=True)

In [ ]:
discriminator.summary()

In [ ]:
rmse_list = list()
psnr_list = list()

In [ ]:
import random
def generate_and_save_images(model, epoch, test_inputs, test_labels):
    if model is None:
        predictions = test_inputs
    else:
        # Make sure the training parameter is set to False because we
        # don't want to train the batchnorm layer when doing inference.
        predictions = model(test_inputs, training=False)

    types = [predictions, test_labels]  # Image types (alternated in rows)
    ntype = len(types)
    nrows = 4
    ncols = 8
    fig = plt.figure(figsize=(8, 5))
    
    for i in range(ntype * predictions.shape[0]):
        plt.subplot(nrows, ncols, i+1)
        # Get relative index
        row = int(i / ncols)
        row_rel = row % ntype
        group = int(row / ntype)
        shift = ncols * (group * (ntype - 1) + row_rel)
        idx = i - shift
        # Plot
        for t in range(ntype):
            if row_rel == 0:
                j = int(i / ntype)
                rmse_ = rmse(test_labels[j], predictions[j], norm=2)
                psnr_ = psnr(test_labels[j], predictions[j], max_diff=1)

                rmse_list.append(rmse_)
                psnr_list.append(psnr_)
                
                plt.xlabel('RMSE={:.3f}\nPSNR={:.2f}'.format(rmse_, psnr_), fontsize=8)
            if row_rel == t:
                plt.imshow(types[row_rel][idx, :, :, 0], vmin=-1, vmax=1, cmap='gray')
                break
        plt.xticks([])
        plt.yticks([])
    
    plt.savefig(os.path.join('/content/drive/MyDrive/output_images/', 'image_at_epoch_{:04d}.png'.format(epoch)))

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return real_loss, fake_loss, total_loss

In [ ]:
def generator_d_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

In [ ]:
def generator_abs_loss(labels, generated_images):
    return config.L1_lambda * tf.compat.v1.losses.absolute_difference(labels, generated_images)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# create a line plot of loss for the gan and save to file
import matplotlib.pyplot as plt

def plot_history(d1_hist, d2_hist, g_hist):
	# plot loss
 plt.subplot(2, 1, 1)
 plt.plot(d1_hist, label='d-real')
 plt.plot(d2_hist, label='d-fake')
 plt.plot(g_hist, label='gen')
 plt.legend()
 plt.show()


In [ ]:
d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()

In [ ]:
# This annotation causes the function to be "compiled".
# @tf.function
def train_step(inputs, labels, epoch):


    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator_large(inputs, training=True)

      real_output = discriminator(labels, training=True)
      generated_output = discriminator(generated_images, training=True)

      gen_d_loss = generator_d_loss(generated_output)
      gen_abs_loss = generator_abs_loss(labels, generated_images)
      gen_loss = gen_d_loss + gen_abs_loss


      gen_rmse = rmse(labels, generated_images)
      gen_psnr = psnr(labels, generated_images)
      real_loss, fake_loss, disc_loss = discriminator_loss(real_output, generated_output)
      
      d1_hist.append(real_loss)
      d2_hist.append(fake_loss)
      g_hist.append(gen_loss)

      print("Generator loss: ",gen_loss)
      print("Discriminator loss: ", disc_loss)


    gradients_of_generator = gen_tape.gradient(gen_loss, generator_large.trainable_weights)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_weights)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator_large.trainable_weights))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_weights))

In [ ]:
import time
def train(train_dataset, epochs):

  
  for epoch in range(epochs):
    start = time.time()

    num_loop = 0
    for x_train_input, x_train_label in train_dataset:
      if ( num_loop != 5 ):
        train_step(x_train_input, x_train_label, epoch)
      else:
        break
      num_loop += 1

    
    num_examples_to_generate = 16
    random_indices = np.random.choice(np.arange(test_X.shape[0]), num_examples_to_generate, replace=False)

    selected_inputs = test_X[random_indices]
    selected_labels = test_y[random_indices]

    generate_and_save_images(generator_large, epoch + 1,selected_inputs,selected_labels)

    print ('Time taken for epoch =========================================== {} is {} sec'.format(epoch + 1, time.time()-start))

  generate_and_save_images(generator_large,epochs,selected_inputs,selected_labels)
  plot_history(d1_hist, d2_hist, g_hist)

In [ ]:
# train(train_inputs, train_labels, 20) 

train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y)).shuffle(config.buffer_size).batch(config.batch_size)
train(train_dataset, config.max_epoch)

In [ ]:
plot_history(d1_hist, d2_hist, g_hist)

In [ ]:
rmse_list = list()
psnr_list = list()

rmse_array = np.array(rmse_list)


In [ ]:
psnr_array = np.array(psnr_list)

In [ ]:
print("mean of rmse: ", np.mean(psnr_array))
print("mean of psnr: ", np.mean(psnr_array))

In [ ]:
print("mean of rmse: ", np.std(psnr_array))
print("mean of psnr: ", np.std(psnr_array))